# Open Data NYC - MTA Subway EDA

In [9]:
import pandas as pd
import numpy as np
import folium
from configparser import ConfigParser


In [10]:
config = ConfigParser()
config.read('../config.ini')

['../config.ini']

In [15]:
csv_path = config['DATA_SOURCES']['subway_csv']
data = pd.read_csv('../'+csv_path,index_col=0)
data.head(3)

,gtfs_stop_id,station_id,complex_id,division,line,stop_name,borough,cbd,daytime_routes,structure,...,south_direction_label,ada,ada_northbound,ada_southbound,georeference,:@computed_region_yamh_8v7k,:@computed_region_wbg7_3whc,:@computed_region_kjdx_g34t,ada_notes,last_modified
0,R01,1,1,BMT,Astoria,Astoria-Ditmars Blvd,Q,False,N W,Elevated,...,Manhattan,0,0,0,"{'type': 'Point', 'coordinates': [-73.912034, ...",196,877.0,2137.0,NaN,"Thu, 22 Aug 2024 18:09:53 GMT"
1,R03,2,2,BMT,Astoria,Astoria Blvd,Q,False,N W,Elevated,...,Manhattan,1,1,1,"{'type': 'Point', 'coordinates': [-73.917843, ...",196,874.0,2137.0,NaN,"Thu, 22 Aug 2024 18:09:53 GMT"
2,R04,3,3,BMT,Astoria,30 Av,Q,False,N W,Elevated,...,Manhattan,0,0,0,"{'type': 'Point', 'coordinates': [-73.921479, ...",196,874.0,2137.0,NaN,"Thu, 22 Aug 2024 18:09:53 GMT"


In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 496 entries, 0 to 495
Data columns (total 23 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   gtfs_stop_id                 496 non-null    object 
 1   station_id                   496 non-null    int64  
 2   complex_id                   496 non-null    int64  
 3   division                     496 non-null    object 
 4   line                         496 non-null    object 
 5   stop_name                    496 non-null    object 
 6   borough                      496 non-null    object 
 7   cbd                          496 non-null    bool   
 8   daytime_routes               496 non-null    object 
 9   structure                    496 non-null    object 
 10  gtfs_latitude                496 non-null    float64
 11  gtfs_longitude               496 non-null    float64
 12  north_direction_label        496 non-null    object 
 13  south_direction_label    

In [21]:
required_columns = ['gtfs_stop_id','stop_name','borough','gtfs_latitude','gtfs_longitude','north_direction_label','south_direction_label','daytime_routes','structure','ada']
df = data.loc[:,required_columns]
df.head()

,gtfs_stop_id,stop_name,borough,gtfs_latitude,gtfs_longitude,north_direction_label,south_direction_label,daytime_routes,structure,ada
0,R01,Astoria-Ditmars Blvd,Q,40.775036,-73.912034,Last Stop,Manhattan,N W,Elevated,0
1,R03,Astoria Blvd,Q,40.770258,-73.917843,Astoria,Manhattan,N W,Elevated,1
2,R04,30 Av,Q,40.766779,-73.921479,Astoria,Manhattan,N W,Elevated,0
3,R05,Broadway,Q,40.761820,-73.925508,Astoria,Manhattan,N W,Elevated,0
4,R06,36 Av,Q,40.756804,-73.929575,Astoria,Manhattan,N W,Elevated,0


In [ ]:
# initiate map with first point
map = folium.Map(location = [lat,long], zoom_start = 6) # zoom start gives an initial map scale

# add remaining points
for c in coordinate_list:
	map.add_child(folium.Marker(location=c, popup=“site name”, icon = folium.Icon(color=‘blue)))

map.save(“mymap.html”) # save to html